In [1]:
from rosetta import *
rosetta.init('-ignore_unrecognized_res T')
#from sjb_scoreutil import dataframe_from_pose_and_sfxn

In [2]:
from toolbox import mutate_residue
import math
import re

In [3]:
p = pose_from_pdb('/Users/siegel03/Downloads/4zp1.pdb')
sfxn = get_fa_scorefxn()

In [4]:
def relax_by_wilson(pose, protein_length):    
    relax = FastRelax(get_fa_scorefxn())
    mm = MoveMap()
    mm.set_bb_true_range(1,protein_length)
    mm.set_chi_true_range(1,protein_length)
    relax.set_movemap(mm)
    relax.apply(pose)
    return sfxn(pose), pose

def mutate_all_x_to_y(pose,x,y):
    #not ready, need to print delta-energy table, might also need to implement mc
    def repack_around(pose, res):
        mm = MoveMap()
        for i in range(1, pose.total_residue()+1): # loop over protein
            res1 = p.residue(res).nbr_atom_xyz()
            res2 = p.residue(i).nbr_atom_xyz()
            distance = math.sqrt((res1[0]-res2[0])**2 + (res1[1]-res2[1])**2 + (res1[2]-res2[2])**2)
            if distance < 6:
                mm.set_bb(i,1)
                mm.set_chi(i,1)
        relax = FastRelax(get_fa_scorefxn())
        relax.set_movemap(mm)
        relax.apply(pose)
    
    res_num = 1
    for i in pose.sequence():    
        if i == x:
            mutant = mutate_residue(pose, res_num, y)
            repack_around(mutant, res_num)
            mutant.dump_pdb('./%s%s%s.pdb'%(x,res_num,y))
            break
        res_num += 1

def resi_energy(pose, res_num, energy_type):
    # pick energy_type from here:
    # total_score, fa_atr, fa_rep, fa_sol, fa_intra_atr, fa_intra_rep, fa_intra_sol,
    # rama, omega, fa_dun, fa_dun_dev, fa_dun_rot, p_aa_pp, ref    
    energy_table = str(pose.energies().residue_total_energies(res_num))
    energy = float(re.search('%s; (.*)\) \( '%energy_type, energy_table).group(1))
    return energy

In [14]:
def mutate_x_to_y(pose,x,y):
    
    def repack_around(pose, res):
        mm = MoveMap()
        for i in range(1, pose.total_residue()+1): # loop over protein
            res1 = p.residue(res).nbr_atom_xyz()
            res2 = p.residue(i).nbr_atom_xyz()
            distance = math.sqrt((res1[0]-res2[0])**2 + (res1[1]-res2[1])**2 + (res1[2]-res2[2])**2)
            if distance < 6:
                mm.set_bb(i,1)
                mm.set_chi(i,1)
        relax = FastRelax(get_fa_scorefxn())
        relax.set_movemap(mm)
        relax.apply(pose)
        
    def resi_energy(pose, res_num, energy_type):
        # pick energy_type from here:
        # total_score, fa_atr, fa_rep, fa_sol, fa_intra_atr, fa_intra_rep, fa_intra_sol,
        # rama, omega, fa_dun, fa_dun_dev, fa_dun_rot, p_aa_pp, ref    
        energy_table = str(pose.energies().residue_total_energies(res_num))
        energy = float(re.search('%s; (.*)\) \( '%energy_type, energy_table).group(1))
        return energy
        
    sfxn = get_fa_scorefxn()    
    mutant = mutate_residue(pose, x, y)
    repack_around(mutant, x)
    mutant.dump_pdb('./%s%s%s.pdb'%(pose.sequence()[x-1],x,y))
    pose_deltaE = sfxn(mutant) - sfxn(pose)
    res_deltaE  = resi_energy(mutant, x, 'total_score') - resi_energy(pose, x, 'total_score')
    
    print '%s%s%s Resi_DeltaE: %s, pose_DeltaE: %s'%(pose.sequence()[x-1],x,y,res_deltaE,pose_deltaE)

In [16]:
relaxed_pose = relax_by_wilson(p, 566)

In [62]:
relaxed_pose.dump_pdb('./test.pdb')

True

In [75]:
def pmut_by_wilson():
    
    mut_dict = {'V':['I','F','Y'],
            'I':['V','L','F','Y'],
            'L':['V','W','F','Y','I'],
            'F':['L','W','Y'],
            'W':['F','Y'],
#            'D':['E','N','Q'],
#            'E':['D','N','Q'],
#            'N':['D','E','Q'],
#            'Q':['D','E','N'],
#            'A':['S','E','N'],
#            'S':['T','N'],
               }

    def mutate_x_to_y(pose,x,y):
    
        def repack_around(pose, res):
            mm = MoveMap()
            for i in range(1, pose.total_residue()+1): # loop over protein
                res1 = p.residue(res).nbr_atom_xyz()
                res2 = p.residue(i).nbr_atom_xyz()
                distance = math.sqrt((res1[0]-res2[0])**2 + (res1[1]-res2[1])**2 + (res1[2]-res2[2])**2)
                if distance < 8:
                    mm.set_bb(i,1)
                    mm.set_chi(i,1)
            relax = FastRelax(get_fa_scorefxn())
            relax.set_movemap(mm)
            relax.apply(pose)
        
        def resi_energy(pose, res_num, energy_type):
            # pick energy_type from here:
            # total_score, fa_atr, fa_rep, fa_sol, fa_intra_atr, fa_intra_rep, fa_intra_sol,
            # rama, omega, fa_dun, fa_dun_dev, fa_dun_rot, p_aa_pp, ref    
            energy_table = str(pose.energies().residue_total_energies(res_num))
            energy = float(re.search('%s; (.*)\) \( '%energy_type, energy_table).group(1))
            return energy

        sfxn = get_fa_scorefxn()    
        mutant = mutate_residue(pose, x, y)
        repack_around(pose, x)
        repack_around(mutant, x)
        pose_deltaE = sfxn(mutant) - sfxn(pose)
        res_deltaE  = resi_energy(mutant, x, 'total_score') - resi_energy(pose, x, 'total_score')
        if res_deltaE < 0.1 and pose_deltaE < 0.1:
            mutant.dump_pdb('./%s%s%s.pdb'%(pose.sequence()[x-1],x,y))
            print '%s%s%s Resi_DeltaE: %s, pose_DeltaE: %s'%(pose.sequence()[x-1],x,y,res_deltaE,pose_deltaE)

            
    for native_res in mut_dict.iterkeys():  
        res_num = 1
        for i in relaxed_pose.sequence():
            if i == native_res:
                for mut_res in mut_dict[native_res]:
                    mutate_x_to_y(relaxed_pose, res_num, mut_res)
                res_num += 1
            else:
                res_num += 1

In [ ]:
pmut_by_wilson()

In [72]:
def repack_around(pose, res):
    mm = MoveMap()
    for i in range(1, pose.total_residue()+1): # loop over protein
        res1 = p.residue(res).nbr_atom_xyz()
        res2 = p.residue(i).nbr_atom_xyz()
        distance = math.sqrt((res1[0]-res2[0])**2 + (res1[1]-res2[1])**2 + (res1[2]-res2[2])**2)
        if distance < 8:
            mm.set_bb(i,1)
            mm.set_chi(i,1)
    relax = FastRelax(get_fa_scorefxn())
    relax.set_movemap(mm)
    relax.apply(pose)

In [73]:
repack_around(p, 5)

None


In [240]:
from rosetta.protocols.enzdes import *

In [241]:
enz = EnzRepackMinimize

In [243]:
enz.

In [1]:
p.sequence()

NameError: name 'p' is not defined

In [47]:
mut_dict = {'V':['I','F','Y'],
            'I':['V','L','F','Y'],
            'L':['V','W','F','Y','I'],
            'F':['L','W','Y'],
            'W':['F','Y'],
            'D':['E','N','Q'],
            'E':['D','N','Q'],
            'N':['D','E','Q'],
            'Q':['D','E','N'],
            'A':['S','E','N'],
            'S':['T','N'],}

In [54]:
for native_res in mut_dict.iterkeys():  
    res_num += 1
    for i in relaxed_pose.sequence():
        if i == native_res:
            for mut_res in mut_dict[native_res]:
                mutate_x_to_y(relaxed_pose, res_num, mut_res)
            res_num += 1
        else:
            res_num += 1

['I', 'F', 'Y']

In [4]:
relaxed_energy = relax_by_wilson(p, 566)[0]
relaxed_pose = relax_by_wilson(p, 566)[1]

-459.940699469
-1299.74729751
-1299.74729751
-1299.0306019


In [28]:
relaxed_pose_1 = relaxed_pose

In [209]:
for i in str(p.energies().residue_total_energies(4)).split(') ( '):
    print i

( fa_atr; -5.90036
fa_rep; 0.751524
fa_sol; 2.31446
fa_intra_atr; -0.421255
fa_intra_rep; 3.96128
fa_intra_sol; 0.443575
fa_intra_atr_xover4; 0
fa_intra_rep_xover4; 0
fa_intra_sol_xover4; 0
fa_intra_RNA_base_phos_atr; 0
fa_intra_RNA_base_phos_rep; 0
fa_intra_RNA_base_phos_sol; 0
fa_atr_dummy; 0
fa_rep_dummy; 0
fa_sol_dummy; 0
lk_hack; 0
lk_ball; 0
lk_ball_wtd; 0
lk_ball_iso; 0
coarse_fa_atr; 0
coarse_fa_rep; 0
coarse_fa_sol; 0
coarse_beadlj; 0
mm_lj_intra_rep; 0
mm_lj_intra_atr; 0
mm_lj_inter_rep; 0
mm_lj_inter_atr; 0
mm_twist; 0
mm_bend; 0
mm_stretch; 0
lk_costheta; 0
lk_polar; 0
lk_nonpolar; 0
lk_polar_intra_RNA; 0
lk_nonpolar_intra_RNA; 0
fa_elec; -0.498405
fa_elec_bb_bb; 0
fa_elec_bb_sc; 0
fa_elec_sc_sc; 0
h2o_hbond; 0
dna_dr; 0
dna_bp; 0
dna_bs; 0
peptide_bond; 0
pcs; 0
pcsTs1; 0
pcsTs2; 0
pcsTs3; 0
pcsTs4; 0
pcs2; 0
fastsaxs; 0
saxs_score; 0
saxs_cen_score; 0
saxs_fa_score; 0
pddf_score; 0
fiberdiffraction; 0
fiberdiffractiondens; 0
fa_mbenv; 0
fa_mbsolv; 0
fa_elec_rna_phos_phos;

In [117]:
mutate_resi_from_x_to_y(relaxed_pose,'V','I')

In [121]:
print p.residue(4)

Residue 4: VAL (VAL, V):
 Properties: POLYMER PROTEIN HYDROPHOBIC METALBINDING ALPHA_AA L_AA
 Variant types:
 Main-chain atoms:  N    CA   C  
 Backbone atoms:    N    CA   C    O    H    HA 
 Ring atoms:  
 Side-chain atoms:  CB   CG1  CG2  HB  1HG1 2HG1 3HG1 1HG2 2HG2 3HG2
 Atom Coordinates:
   N  : 30.8146, 7.66751, 125.654
   CA : 29.4891, 7.33493, 125.139
   C  : 28.5356, 6.94896, 126.269
   O  : 27.4247, 7.47876, 126.382
   CB : 29.5846, 6.17327, 124.132
   CG1: 28.1962, 5.66972, 123.767
   CG2: 30.3386, 6.6248, 122.891
   H  : 31.6226, 7.19429, 125.275
   HA : 29.0864, 8.21066, 124.629
   HB : 30.1151, 5.34302, 124.599
  1HG1: 28.2826, 4.84864, 123.055
  2HG1: 27.6886, 5.31835, 124.665
  3HG1: 27.622, 6.47973, 123.317
  1HG2: 30.403, 5.79834, 122.184
  2HG2: 29.811, 7.45888, 122.428
  3HG2: 31.3435, 6.94166, 123.17



In [108]:
def repack_around(pose, res):
    mm = MoveMap()
    for i in range(1, pose.total_residue()+1): # loop over protein
        res1 = p.residue(res).nbr_atom_xyz()
        res2 = p.residue(i).nbr_atom_xyz()
        distance = math.sqrt((res1[0]-res2[0])**2 + (res1[1]-res2[1])**2 + (res1[2]-res2[2])**2)
        if distance < 6:
            mm.set_bb(i,1)
            mm.set_chi(i,1)
    relax = FastRelax(get_fa_scorefxn())
    relax.set_movemap(mm)
    relax.apply(pose)
    print mm

In [109]:
repack_around(p,5)


-------------------------------
  resnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT      BB     FALSE
 DEFAULT      SC     FALSE
 DEFAULT      NU     FALSE
 DEFAULT  BRANCH     FALSE
     003      BB      TRUE
              SC      TRUE
     004      BB      TRUE
              SC      TRUE
     005      BB      TRUE
              SC      TRUE
     006      BB      TRUE
              SC      TRUE
     007      BB      TRUE
              SC      TRUE
     008      BB      TRUE
              SC      TRUE
     009      BB      TRUE
              SC      TRUE
     031      BB      TRUE
              SC      TRUE
     034      BB      TRUE
              SC      TRUE
     170      BB      TRUE
              SC      TRUE
-------------------------------
 jumpnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT     JUMP    FALSE
-------------------------------
  resnum  atomnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT               PHI    

In [ ]:
pro_mut_score_df_list = list()

pack_radius = 10

aa_bool = rosetta.utility.vector1_bool()
mutant_aa = aa_from_oneletter_code('P')
for i in range(1,21):
    aa_bool.append(i==mutant_aa) ## Clever method, credit to Evan Baugh

# make mutations over whole protein
#for i in range(1,p.total_residue()+1):
for i in range(1,11):

    print "Mutating Residue %s " %i

    #make mutations on test pose
    test_pose = Pose()
    test_pose.assign(p)
    task = standard_packer_task(test_pose)
    task.nonconst_residue_task(i).restrict_absent_canonical_aas( aa_bool ) #sets to just Proline
    center = test_pose.residue(i).nbr_atom_xyz() # center of residue to mutate

    testmm = MoveMap()
    
   
    for j in range(1, test_pose.total_residue()+1): # loop over protein
    # if not the mutant site, or within the pack radius, set i to no repack
        if not j == i:
            repack_this_dist = center.distance_squared( test_pose.residue(j).nbr_atom_xyz() ) 
            #print j,repack_this_dist
            
            if not (repack_this_dist  > pack_radius**2): 
                #allow repack
                task.nonconst_residue_task(j).restrict_to_repacking()
                
                #allow bb movement in movemap
                testmm.set_bb(j,1) #Boolean True = the int 1
                testmm.set_chi(j,1)
                #print "Keeping resi %s" %j
                
            else:
            #print "Preventing Packing at resi %s" %i
                task.nonconst_residue_task(j).prevent_repacking()

In [71]:
mm = MoveMap()
?mm.set_ranges_unmodifiable()
print mm


-------------------------------
  resnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT      BB     FALSE
 DEFAULT      SC     FALSE
 DEFAULT      NU     FALSE
 DEFAULT  BRANCH     FALSE
-------------------------------
 jumpnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT     JUMP    FALSE
-------------------------------
  resnum  atomnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT               PHI    FALSE
 DEFAULT             THETA    FALSE
 DEFAULT                 D    FALSE
 DEFAULT               RB1    FALSE
 DEFAULT               RB2    FALSE
 DEFAULT               RB3    FALSE
 DEFAULT               RB4    FALSE
 DEFAULT               RB5    FALSE
 DEFAULT               RB6    FALSE




In [45]:
print sfxn(p)
after = p

-985.716848605
<class 'rosetta.core.pose.Pose'>


In [47]:
pymol = PyMOLMover()
pymol.apply(p)

In [50]:
while True:
    pymol.apply(p)
    pymol.send_energy(p)
    pymol.apply(after)
    pymol.send_energy(after)

KeyboardInterrupt: 

In [57]:
pymol.apply(p)
sfxn(p)
pymol.send_energy(p)
pymol.label_energy(p)

In [58]:
pymol.apply(after)
sfxn(after)
pymol.send_energy(after)
pymol.label_energy(after)

In [28]:
sfxn = get_fa_scorefxn()
print sfxn(p)
task = standard_packer_task(p)
print task

-127.369073109
#Packer_Task

resid	pack?	design?	allowed_aas
1	TRUE	TRUE	ALA:NtermProteinFull,CYS:NtermProteinFull,ASP:NtermProteinFull,GLU:NtermProteinFull,PHE:NtermProteinFull,GLY:NtermProteinFull,HIS:NtermProteinFull,HIS_D:NtermProteinFull,ILE:NtermProteinFull,LYS:NtermProteinFull,LEU:NtermProteinFull,MET:NtermProteinFull,ASN:NtermProteinFull,PRO:NtermProteinFull,GLN:NtermProteinFull,ARG:NtermProteinFull,SER:NtermProteinFull,THR:NtermProteinFull,VAL:NtermProteinFull,TRP:NtermProteinFull,TYR:NtermProteinFull
2	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
3	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
4	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
5	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
6	TRUE	TRUE	ALA,CYS,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TR

In [29]:
relax = MinMover()
move_map = MoveMap()
relax.score_function(sfxn)
relax.movemap(move_map)

In [33]:
move_map.set_bb(1)
move_map.set_sc(1)
print move_map


-------------------------------
  resnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT      BB      TRUE
 DEFAULT      SC     FALSE
 DEFAULT      NU     FALSE
 DEFAULT  BRANCH     FALSE
-------------------------------
 jumpnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT     JUMP    FALSE
-------------------------------
  resnum  atomnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT               PHI    FALSE
 DEFAULT             THETA    FALSE
 DEFAULT                 D    FALSE
 DEFAULT               RB1    FALSE
 DEFAULT               RB2    FALSE
 DEFAULT               RB3    FALSE
 DEFAULT               RB4    FALSE
 DEFAULT               RB5    FALSE
 DEFAULT               RB6    FALSE




In [170]:
fa_attr = core.scoring.ScoreType.fa_atr
fa = core.scoring.ScoreTypeManager.score_type_from_name('fa_atr')

#print res1energies.get(fa)

print sfxn.get_nonzero_weighted_scoretypes()
class PyRosettaScore(object):
    def __init__(self,sfxn):
        for i in sfxn.get_nonzero_weighted_scoretypes():
        #for i in args:
            setattr(self,str(i),core.scoring.ScoreTypeManager.score_type_from_name('%s' %i ))  
            
#scoretypes = PyRosettaScore(sfxn)
#print scoretypes

[fa_atr, fa_rep, fa_sol, fa_intra_rep, fa_elec, pro_close, hbond_sr_bb, hbond_lr_bb, hbond_bb_sc, hbond_sc, dslf_fa13, rama, omega, fa_dun, p_aa_pp, ref, ]


In [5]:
import pandas as pd

In [6]:
pro_mut_score_df_list = list()

pack_radius = 10

aa_bool = rosetta.utility.vector1_bool()
mutant_aa = aa_from_oneletter_code('P')
for i in range(1,21):
    aa_bool.append(i==mutant_aa) ## Clever method, credit to Evan Baugh


pym = PyMOLMover()
pym.keep_history(True)
pym.update_energy(True)

# make mutations over whole protein
#for i in range(1,p.total_residue()+1):
for i in range(1,11):

    print "Mutating Residue %s " %i

    #make mutations on test pose
    test_pose = Pose()
    test_pose.assign(p)
    task = standard_packer_task(test_pose)
    task.nonconst_residue_task(i).restrict_absent_canonical_aas( aa_bool ) #sets to just Proline
    center = test_pose.residue(i).nbr_atom_xyz() # center of residue to mutate

    testmm = MoveMap()
    
   
    for j in range(1, test_pose.total_residue()+1): # loop over protein
    # if not the mutant site, or within the pack radius, set i to no repack
        if not j == i:
            repack_this_dist = center.distance_squared( test_pose.residue(j).nbr_atom_xyz() ) 
            #print j,repack_this_dist
            
            if not (repack_this_dist  > pack_radius**2): 
                #allow repack
                task.nonconst_residue_task(j).restrict_to_repacking()
                
                #allow bb movement in movemap
                testmm.set_bb(j,1) #Boolean True = the int 1
                testmm.set_chi(j,1)
                #print "Keeping resi %s" %j
                
            else:
            #print "Preventing Packing at resi %s" %i
                task.nonconst_residue_task(j).prevent_repacking()
    
    
#print task
    packer = PackRotamersMover(sfxn, task)
    sfxn(test_pose)
    pym.apply(test_pose)    
    print "Native pose energy for residue %s" %i
    pre_mut_resi =  p.energies().residue_total_energies(i).sum()

    packer.apply(test_pose)
    #sfxn(test_pose)
    pym.apply(test_pose)
    #print test_pose
    print "Mutation to Proline at resi %s has energies: " %i
    post_mut_resi = test_pose.energies().residue_total_energies(i).sum()

  

    #### Doesn't Correctly include the hbond terms, need to fix
    
    for scoreterm in [ a for a in dir(scoretypes) if not a.startswith('__')]:
        #print type(getattr(scoretypes,scoreterm))
        #print scoreterm
        refenergy =  p.energies().residue_total_energies(i).get( getattr(scoretypes,scoreterm) )
        testenergy = test_pose.energies().residue_total_energies(i).get( getattr(scoretypes,scoreterm) )
        diff = testenergy - refenergy
        #print "Native: %s, Pro: %s, Delta: %s" %(refenergy, testenergy, diff)
        print "%s Change: %s" %(scoreterm, diff)
        ## same same 
## print "----- %s" %p.energies().residue_total_energies(i)[getattr(scoretypes,scoreterm)]
        ## just store and use the DF to make the deltas

    minmover = MinMover()
    minmover.score_function(sfxn)
    minmover.movemap(testmm)
#    minmover.apply(test_pose)
    
    refposephi = p.phi(i)
    refposepsi = p.psi(i)
    refposeomega = p.omega(i)
    
    ### insert movemap and backbone movents here
    print "Pre-minmover score"
    print sfxn(test_pose)
    minmover.apply(test_pose)
    print "pose minmover score"
    print sfxn(test_pose)
    
    testposephi = test_pose.phi(i)
    testposepsi = test_pose.psi(i)
    testposeomega = test_pose.omega(i)
    
    
    ### Also add in the ex1 ex2 flags 
    ###
    ### Now add a mc with the repack and minmover in cycles for each mutant
    ### Recover the low energy one, use the task operation (sets repack shell)
    ## and use movemap for both the repack and the minmover
    ##
    ##
    
    print "**"*30
    print "Previous Phi: %s Psi: %s Omega: %s" %(refposephi,refposepsi,refposeomega)
    print "New Phi: %s Psi: %s Omega: %s" %(testposephi,testposepsi,testposeomega)
    print "**"*30
    
    delta_E = post_mut_resi - pre_mut_resi
    if delta_E < 0:
        print "Resi %s -> Proline has a deltaE of %s" %(i,delta_E)
    

Mutating Residue 1 
Native pose energy for residue 1
Mutation to Proline at resi 1 has energies: 
dslf_fa13 Change: 0.0
fa_atr Change: 0.299905707627
fa_dun Change: -3.49881958926
fa_elec Change: -0.297289991939
fa_intra_rep Change: 0.209014887784
fa_rep Change: 0.0944640658194
fa_sol Change: -0.22493675078
hbond_bb_sc Change: 0.0
hbond_lr_bb Change: 0.0
hbond_sc Change: 0.0
hbond_sr_bb Change: 0.0
omega Change: 0.0
p_aa_pp Change: 0.0
pro_close Change: 1.06097481028
rama Change: 0.0
ref Change: 0.722695
Pre-minmover score
734.117283696
pose minmover score
729.903455389
************************************************************
Previous Phi: 0.0 Psi: 77.5345111646 Omega: -178.980788671
New Phi: 0.0 Psi: 77.5345111646 Omega: -178.980788671
************************************************************
Resi 1 -> Proline has a deltaE of -5.75813376892
Mutating Residue 2 
Native pose energy for residue 2
Mutation to Proline at resi 2 has energies: 
dslf_fa13 Change: 0.0
fa_atr Change: 0.47

In [25]:
#print testmm
#print task

In [50]:
phis = [ p.phi(i) for i in range(1, p.total_residue() + 1)]
psis = [p.psi(i) for i in range(1, p.total_residue() + 1)]
omegas = [p.omega(i) for i in range(1, p.total_residue() + 1)]

phis = {}
psis = {}
omegas = {}
for i in range(1,p.total_residue()+1):
    phis[i] = p.phi(i)
    psis[i] = p.psi(i)
    omegas[i] = p.omega(i)

    
for i in range(1,p.total_residue()+1):
    chis = [p.chi(j + 1, i) for j in range(p.residue(i).nchi() )]
    for chi_no in range(len(chis)):
        print 'Chi ' + str(chi_no + 1) + ':', chis[chi_no]


Chi 1: -176.263166314
Chi 2: 45.6862042589
Chi 1: 177.113934121
Chi 2: 3.39999995495e-05
Chi 1: -50.8855864587
Chi 2: 75.4193179015
Chi 1: -80.8752544668
Chi 2: 65.332471062
Chi 1: -72.6073688698
Chi 2: 74.0864880668
Chi 1: -35.796380359
Chi 2: 45.3741988602
Chi 3: -37.0959845725
Chi 1: 41.3331949565
Chi 2: 3.40000003747e-05
Chi 1: -158.383484626
Chi 2: 69.443778386
Chi 1: -33.6065973493
Chi 2: -162.552647545
Chi 3: -59.9861136752
Chi 1: -67.6829477555
Chi 2: 104.381372542
Chi 1: -178.855576137
Chi 2: 3.40000002282e-05
Chi 1: -68.4016080312
Chi 2: 4.98997728636e-13
Chi 1: 64.5798106661
Chi 2: 3.39999998992e-05
Chi 1: 152.565896639
Chi 2: -1.24077346532e-13
Chi 1: 64.3547127123
Chi 2: -9.92618772256e-15
Chi 1: -171.484640478
Chi 2: 76.2338582205
Chi 3: -6.35623574566e-13
Chi 1: -63.7059676074
Chi 2: 176.922944033
Chi 3: -97.6515959923
Chi 1: 74.2582101463
Chi 2: 177.557749936
Chi 1: -82.610502809
Chi 2: -176.317713251
Chi 3: 22.317512782
Chi 1: 50.0984082486
Chi 2: 3.40000006411e-05
Chi

In [53]:
DSSP = DsspMover()
DSSP.apply(p)    # populates the pose's Pose.secstruct                                                                       
ss = p.secstruct()
print 'Secondary Structure:\n' + ss

Secondary Structure:
LLLELLLLLEEEEELLHHHHLLLLLLLLLLLEHHHHHHHLLLLLHHHLLLLLLLLHHHHHHHHHHHHHHHLLLEEEEELLHHHHLLLLLLLLHHHHHHHHHHHHHHHHHLLEEEEEEELLLLEHHHHHLLHHHLHHHHHHHHHHHHHHHHHLLLLLLEEEEEELHHHHHHHHHHLLLLLLLLLLHHHHHHHHHHHHHHHHHHHHHHHHLLLLLEEEEEEELLLEEELLLLHHHHHHHHHHHHHHHHHHHHHHHHLLLLHHHHHHHHHHHLLLLLLLLLHHHHHLLLLLEEEEELLLLEEEEELLLLLLLLEEEELLLLLELLLLLELLHHHHHHHHHHHHHLLLLLLLEEEEEELLLLLLLLELLEELLHHHHHHHHHHHHHHHHHLLLLLLEEEEEEELLELLLLHHHHHHLLLLLEEELLLLLLEEEEHHHHHHHHHHHHLEL


In [18]:
movemaps = MoveMap()
print movemaps
movemaps.set_bb(1,1)
movemaps.set_bb(2,1)
print movemaps


-------------------------------
  resnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT      BB     FALSE
 DEFAULT      SC     FALSE
 DEFAULT      NU     FALSE
 DEFAULT  BRANCH     FALSE
-------------------------------
 jumpnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT     JUMP    FALSE
-------------------------------
  resnum  atomnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT               PHI    FALSE
 DEFAULT             THETA    FALSE
 DEFAULT                 D    FALSE
 DEFAULT               RB1    FALSE
 DEFAULT               RB2    FALSE
 DEFAULT               RB3    FALSE
 DEFAULT               RB4    FALSE
 DEFAULT               RB5    FALSE
 DEFAULT               RB6    FALSE



-------------------------------
  resnum     Type  TRUE/FALSE 
-------------------------------
 DEFAULT      BB     FALSE
 DEFAULT      SC     FALSE
 DEFAULT      NU     FALSE
 DEFAULT  BRANCH     FALSE
     001      BB      TRUE
     002     

In [36]:
perressasa = core.scoring.sasa.per_res_sc_sasa(p)

In [39]:
count = 1
for i in perressasa:
    print count, i
    count +=1
#print perressasa


1 67.4303302297
2 80.6181156279
3 28.4526832608
4 0.0
5 0.0
6 21.1065846821
7 0.0
8 1.28203354131
9 0.0
10 22.0549500556
11 0.0
12 3.33140690602
13 23.5818959545
14 24.1705606046
15 0.0
16 0.0
17 0.76969020013
18 132.342654555
19 40.8740131358
20 1.02468668236
21 0.0
22 0.0
23 2.30672022366
24 0.0
25 0.0
26 26.2542044788
27 6.65300118313
28 0.0
29 8.19749345884
30 27.1565474591
31 1.02468668236
32 0.0
33 43.234318846
34 3.91651884148
35 0.512343341178
36 38.4304513418
37 100.950441226
38 9.6262510126
39 180.597533228
40 71.9529049576
41 0.0
42 98.205705416
43 7.17280677649
44 5.63577675295
45 35.621673584
46 0.0
47 0.0
48 8.31432967748
49 10.3385547869
50 11.2739038826
51 0.446804288511
52 0.0
53 0.0
54 0.446804288511
55 0.0
56 0.0
57 0.0
58 0.0
59 0.446804288511
60 0.0
61 44.6690027622
62 0.512343341178
63 58.999366016
64 0.0
65 2.05172374144
66 0.0
67 1.02468668236
68 0.0
69 0.76969020013
70 0.0
71 1.53703002353
72 3.07876080052
73 0.0
74 10.7639109182
75 0.0
76 0.0
77 0.0
78 3.64890